# Quantization of `granite-3.3-2b-instruct` model

Recall that our overall solution uses the quantized version of the model `granite-3.3-2b-instruct`. In this lab, we will be taking in the base model `granite-3.3-2b-instruct` and quantizing it to `W4A16` - which is fixed-point integer (INT) quantization scheme for weights and floating‑point for activations - to provide both memory savings (weight - INT4) and inference acceleration (activations - BF16) with `vLLM`

**Note**: `W4A16` computation is supported on Nvidia GPUs with compute capability > 7.5 (Turing, Ampere, Ada Lovelace, Hopper).

**Note**: The steps here will take around 20-30 minutes, depending on the connectivity. The most time consuming steps are the installation of llmcompressor (up to 5 mins) and the quantization step (which can take more anywhere between 10-15 mins)

## Setting up llm-compressor

Installing `llmcompressor` may take a minute, depending on the bandwith available. Do note the versions of `transformer` library we would be using. There is a known issue (*torch.fx.proxy.TraceError: symbolically traced variables cannot be used as inputs to control flow*) with the usage of the latest transformer library (version `4.53.2` as of July 17, 2025) in combination with the latest version of llmcompressor (version `0.6.0`).

In [1]:
!pip install -q llmcompressor==0.6.0 transformers==4.52.2 boto3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
vllm 0.10.0 requires transformers>=4.53.2, but you have transformers 4.52.2 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


Let's make sure we have installed the right versions installed

In [2]:
!pip list | grep llmcompressor

llmcompressor                     0.6.0


In [3]:
!pip list | grep transformer

transformers                      4.52.2


## Let' start with the quantization of the model

There are 6 steps:
1. Loading the model
2. Choosing the quantization scheme and method
3. Preparing the calibration data
4. Applying quantization
5. Saving the model
6. Evaluation of accuracy in vLLM

### Loading the model

First, let's download the model from local S3 and then load the model using AutoModelForCausalLM for handling quantized saving and loading. 

In [4]:
import os
from boto3 import client

def download_model_from_s3(s3_path, local_model_dir):
    print('Starting download of model from S3')
    
    # Get S3 credentials from environment
    s3_endpoint_url = os.environ["AWS_S3_ENDPOINT"]
    s3_access_key = os.environ["AWS_ACCESS_KEY_ID"]
    s3_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
    s3_bucket_name = os.environ["AWS_S3_BUCKET"]

    print(f'Downloading model from bucket {s3_bucket_name} '
          f'path {s3_path} from S3 storage at {s3_endpoint_url}')
    print(f'Target local directory: {local_model_dir}')

    s3_client = client(
        's3', endpoint_url=s3_endpoint_url, aws_access_key_id=s3_access_key,
        aws_secret_access_key=s3_secret_key, verify=False
    )

    os.makedirs(local_model_dir, exist_ok=True)

    print(f'Listing objects with prefix: {s3_path}')
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=s3_bucket_name, Prefix=s3_path)

    downloaded_files = []
    total_objects = 0
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                total_objects += 1
                s3_key = obj['Key']
                print(f'Found S3 object: {s3_key}')
                
                # Skip if it's just a directory marker
                if s3_key.endswith('/'):
                    print(f'Skipping directory marker: {s3_key}')
                    continue
                
                relative_path = s3_key[len(s3_path):].lstrip('/')
                local_file_path = os.path.join(local_model_dir, relative_path)
                
                print(f'S3 key: {s3_key}')
                print(f'Local file path: {local_file_path}')
                
                local_dir = os.path.dirname(local_file_path)
                if local_dir:
                    os.makedirs(local_dir, exist_ok=True)
                    print(f'Created directory: {local_dir}')
                
                try:
                    s3_client.download_file(s3_bucket_name, s3_key, local_file_path)
                    downloaded_files.append(local_file_path)
                    print(f'Successfully downloaded {s3_key} to {local_file_path}')
                except Exception as e:
                    print(f'Error downloading {s3_key}: {e}')
                    raise

    print(f'Total objects found: {total_objects}')
    print(f'Files downloaded: {len(downloaded_files)}')
    
    config_path = os.path.join(local_model_dir, 'config.json')
    if os.path.exists(config_path):
        print(f'Verified: config.json found at {config_path}')
    else:
        print(f'Warning: config.json not found at {config_path}')
    
    print('Finished downloading model from S3')
    return local_model_dir

In [5]:
current_dir = os.getcwd()
BASE_MODEL_DIR = current_dir + "/granite-3.3-2b-instruct"
S3_MODEL_PATH = "ibm-granite/granite-3.3-2b-instruct/"  

model_path = download_model_from_s3(S3_MODEL_PATH, BASE_MODEL_DIR)

# use the downloaded model path for tokenizer and model loading
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path, device_map="auto", torch_dtype="auto"
)

Starting download of model from S3
Target local directory: /opt/app-root/src/granite-3.3-2b-instruct
Listing objects with prefix: ibm-granite/granite-3.3-2b-instruct/
Found S3 object: ibm-granite/granite-3.3-2b-instruct/.gitattributes
S3 key: ibm-granite/granite-3.3-2b-instruct/.gitattributes
Local file path: /opt/app-root/src/granite-3.3-2b-instruct/.gitattributes
Created directory: /opt/app-root/src/granite-3.3-2b-instruct
Successfully downloaded ibm-granite/granite-3.3-2b-instruct/.gitattributes to /opt/app-root/src/granite-3.3-2b-instruct/.gitattributes
Found S3 object: ibm-granite/granite-3.3-2b-instruct/README.md
S3 key: ibm-granite/granite-3.3-2b-instruct/README.md
Local file path: /opt/app-root/src/granite-3.3-2b-instruct/README.md
Created directory: /opt/app-root/src/granite-3.3-2b-instruct
Successfully downloaded ibm-granite/granite-3.3-2b-instruct/README.md to /opt/app-root/src/granite-3.3-2b-instruct/README.md
Found S3 object: ibm-granite/granite-3.3-2b-instruct/added_token

/opt/app-root/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.44it/s]


Alternatively, the model can also be loaded from HuggingFace directly as follows:

In [6]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# MODEL_ID = "ibm-granite/granite-3.2-2b-instruct"
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID, device_map="auto", torch_dtype="auto",
# )
# tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

### Prepare calibration data

Prepare the calibration data. When quantizing weigths of a model to int4 using GPTQ, we need some sample data to run the GPTQ algorithms. As a result, it is very important to use calibration data that closely matches the type of data used in our deployment. If you have fine-tuned a model, using a sample of your training data is a good idea.

In our case, we are quantizing an Instruction tuned generic model, so we will use the ultrachat dataset. Some best practices include:
- 512 samples is a good place to start (increase if accuracy drops). We are going to use 256 to speed up the process.
- 2048 sequence length is a good place to start
- Use the chat template or instrucion template that the model is trained with


In [7]:
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES = 512  # 1024
DATASET_ID = "neuralmagic/LLM_compression_calibration"
DATASET_SPLIT = "train"

# Load dataset.
ds = load_dataset(DATASET_ID, split=DATASET_SPLIT)
ds = ds.shuffle(seed=42).select(range(NUM_CALIBRATION_SAMPLES))

# Preprocess the data into the format the model is trained with.
def preprocess(example):
    return {"text": example["text"]}
ds = ds.map(preprocess)

# Tokenize the data
def tokenize(sample):
    return tokenizer(
        sample["text"],
        padding=False,
        truncation=False,
        add_special_tokens=True,
    )
ds = ds.map(tokenize, remove_columns=ds.column_names)

With the dataset ready, we will now apply quantization.

We first select the quantization algorithm. For W4A16, we want to:
- Run SmoothQuant to make the activations easier to quantize
- Quantize the weights to 4 bits with channelwise scales using GPTQ
- Quantize the activations with dynamic per token strategy

**Note**: The quantization step takes a long time to complete due to the callibration requirements -- around 10 - 15 mins, depending on the GPU.

### Imports and definitions

**GPTQModifier**: Applies Gentle Quantization (GPTQ) for weight-only quantization.

**SmoothQuantModifier**: Prepares model activations for smoother quantization by scaling internal activations and weights.

**oneshot**: High-level API that applies your quantization recipe in one go.

In [8]:
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier

### Hyperparameters

Rationale
- **DAMPENING_FRAC=0.1** gently prevents large Hessian-derived updates during quantization.
- **OBSERVER="mse"** measures quantization error by squared deviations, yielding well-rounded scales.
- **GROUP_SIZE=128** determines group size for per-channel quantization; typical default usage.

In [9]:
DAMPENING_FRAC = 0.1  # tapering adjustment to prevent extreme weight updates
OBSERVER = "mse"  # denotes minmax - quantization layout based on mean‐squared‐error
GROUP_SIZE = 128  # # per-channel grouping width for quantization

### Layer Mappings & Ignoring Heads

Logic

- **ignore=["lm_head"]** skips quantization on the output layer to preserve final logits and maintain accuracy.
- mappings link groups of linear projections (q, k, v, gating, up/down projections) with layernorm blocks—SmoothQuant uses these to shift and normalize activations across paired layers for better quant distribution.

In [10]:
ignore=["lm_head"]
mappings=[
    [["re:.*q_proj", "re:.*k_proj", "re:.*v_proj"], "re:.*input_layernorm"],
    [["re:.*gate_proj", "re:.*up_proj"], "re:.*post_attention_layernorm"],
    [["re:.*down_proj"], "re:.*up_proj"]
]

### Recipe Definition

**Workflow**

- **SmoothQuantModifier**: Re-scales activations across paired layers before quantization to reduce outliers (smoothing_strength=0.7, high smoothing but not extreme).
- **GPTQModifier**: Performs Weight-Only quantization (4-bit weights, 16-bit activations) on all Linear layers except those ignored, applying your dampening and observer settings. Scheme "W4A16" reduces model size while maintaining decent accuracy. 

In [11]:
recipe = [
    SmoothQuantModifier(smoothing_strength=0.7, ignore=ignore, mappings=mappings),
    GPTQModifier(
        targets=["Linear"],
        ignore=ignore,
        scheme="W4A16",
        dampening_frac=DAMPENING_FRAC,
        observer=OBSERVER,
    )
]

### Quantize in One Shot

**How It Works**

- Feeds dataset (calibration set) into your model to gather activation statistics.
- Applies SmoothQuant rescaling followed by GPTQ quantization in a sequential per-layer manner.
- **max_seq_length=8196** ensures large context coverage for calibration.

In [12]:
oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
    max_seq_length=8196,
)

2025-08-16T09:15:44.561080+0000 | reset | INFO - Compression lifecycle reset
2025-08-16T09:15:44.564115+0000 | from_modifiers | INFO - Creating recipe from modifiers


/tmp/ipykernel_948/180673739.py:1: DeprecationWarning: `from llmcompressor.transformers import oneshot` is deprecated, please use `from llmcompressor import oneshot`.
  oneshot(


2025-08-16T09:15:45.933448+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-08-16T09:15:45.934151+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `SmoothQuantModifier`


(1/41): Calibrating: 100%|██████████| 512/512 [00:01<00:00, 283.13it/s]

2025-08-16T09:15:50.352219+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.input_layernorm


...

2025-08-16T09:26:40.642091+0000 | compress | METRIC - time 0.83s
2025-08-16T09:26:40.643165+0000 | compress | METRIC - error 59816.77
2025-08-16T09:26:40.643885+0000 | compress | METRIC - GPU 0 | usage: 25.52% | total memory: 24 GB
2025-08-16T09:26:40.644245+0000 | compress | METRIC - Compressed module size: 8.486912 MB
2025-08-16T09:26:40.644962+0000 | compress_modules | INFO - Quantizing model.layers.32.self_attn.k_proj using 512 samples
2025-08-16T09:26:41.432884+0000 | compress | METRIC - time 0.79s
2025-08-16T09:26:41.433904+0000 | compress | METRIC - error 23898.27
2025-08-16T09:26:41.434668+0000 | compress | METRIC - GPU 0 | usage: 25.52% | total memory: 24 GB
2025-08-16T09:26:41.435041+0000 | compress | METRIC - Compressed module size: 2.121728 MB
2025-08-16T09:26:41.435714+0000 | compress_modules | INFO - Quantizing model.layers.32.self_attn.v_proj using 512 samples
2025-08-16T09:26:42.228843+0000 | compress | METRIC - time 0.79s
2025-08-16T09:26:42.229864+0000 | compress | ME

(34/41): Calibrating: 100%|██████████| 512/512 [00:07<00:00, 66.49it/s]

2025-08-16T09:26:56.940557+0000 | compress_modules | INFO - Quantizing model.layers.33.self_attn.q_proj using 512 samples


2025-08-16T09:26:57.765238+0000 | compress | METRIC - time 0.82s
2025-08-16T09:26:57.766257+0000 | compress | METRIC - error 79826.00
2025-08-16T09:26:57.766992+0000 | compress | METRIC - GPU 0 | usage: 25.52% | total memory: 24 GB
2025-08-16T09:26:57.767388+0000 | compress | METRIC - Compressed module size: 8.486912 MB
2025-08-16T09:26:57.768158+0000 | compress_modules | INFO - Quantizing model.layers.33.self_attn.k_proj using 512 samples
2025-08-16T09:26:58.556898+0000 | compress | METRIC - time 0.79s
2025-08-16T09:26:58.557890+0000 | compress | METRIC - error 32549.31
2025-08-16T09:26:58.558550+0000 | compress | METRIC - GPU 0 | usage: 25.52% | total memory: 24 GB
2025-08-16T09:26:58.558938+0000 | compress | METRIC - Compressed module size: 2.121728 MB
2025-08-16T09:26:58.559573+0000 | compress_modules | INFO - Quantizing model.layers.33.self_attn.v_proj using 512 samples
2025-08-16T09:26:59.341812+0000 | compress | METRIC - time 0.78s
2025-08-16T09:26:59.342730+0000 | compress | ME

(35/41): Calibrating:  10%|█         | 53/512 [00:00<00:06, 70.69it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



### Save the Compressed Model

**Explanation**

- Naming: appends -W4A16 to distinguish the quantized checkpoint.
- **save_compressed=True** stores weights in compact safetensors format for deployment via vLLM.

In [13]:
# Save to disk compressed.
MODEL_ID = "ibm-granite/granite-3.2-2b-instruct"
SAVE_DIR = MODEL_ID.split("/")[-1] + "-W4A16"
model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)

2025-08-16T09:28:53.884850+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 527it [00:10, 48.07it/s]


('granite-3.2-2b-instruct-W4A16/tokenizer_config.json',
 'granite-3.2-2b-instruct-W4A16/special_tokens_map.json',
 'granite-3.2-2b-instruct-W4A16/chat_template.jinja',
 'granite-3.2-2b-instruct-W4A16/vocab.json',
 'granite-3.2-2b-instruct-W4A16/merges.txt',
 'granite-3.2-2b-instruct-W4A16/added_tokens.json',
 'granite-3.2-2b-instruct-W4A16/tokenizer.json')

### Evaluate accuracy in vLLM

We can evaluate accuracy with lm_eval

##### Check GPU memory leftovers:

In [14]:
!nvidia-smi

Sat Aug 16 09:29:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.144.03             Driver Version: 550.144.03     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:31:00.0 Off |                    0 |
| N/A   72C    P0             39W /   72W |    5579MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**IMPORTANT**: After quantizing the model the GPU memory may not be freed (see the above output). You need to **restart the kernel** before evaluating the model to ensure you have enough GPU RAM available.

#### Install lm_eval

In [15]:
!pip install -q lm_eval==v0.4.3


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


#### Install vLLM for evaluation

Run the following to test accuracy on GSM-8K:

In [16]:
pip install -q vllm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmcompressor 0.6.0 requires numpy<2.0,>=1.17.0, but you have numpy 2.2.6 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Evaluation Command

- `--model vllm` - Uses vLLM backend for fast, memory-efficient inference on large models 
- `--model_args` - pretrained=$MODEL_ID: specifies which model to load.
- `add_bos_token=true`: ensures a beginning-of-sequence token is added; required for consistent results on math and reasoning tasks 
- `max_model_len=4096`: sets the context window the model uses for evaluation.
- `gpu_memory_utilization=0.5`: limits vLLM to use 50% of GPU memory, allowing to avoid OOM.

In [17]:
import os

current_dir = os.getcwd()

MODEL_ID = current_dir + "/granite-3.2-2b-instruct-W4A16"

!lm_eval --model vllm \
  --model_args "pretrained=$MODEL_ID,add_bos_token=true,max_model_len=4096,gpu_memory_utilization=0.5" \
  --trust_remote_code \
  --tasks gsm8k \
  --num_fewshot 5 \
  --limit 250 \
  --batch_size 'auto'

INFO 08-16 09:29:23 [__init__.py:235] Automatically detected platform cuda.
2025-08-16:09:29:24,368 INFO     [__main__.py:272] Verbosity set to INFO
2025-08-16:09:29:28,939 WARNING  [__main__.py:312]  --limit SHOULD ONLY BE USED FOR TESTING.REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT.
2025-08-16:09:29:28,940 INFO     [__main__.py:357] Passed `--trust_remote_code`, setting environment variable `HF_DATASETS_TRUST_REMOTE_CODE=true`
2025-08-16:09:29:28,940 INFO     [__main__.py:369] Selected Tasks: ['gsm8k']
2025-08-16:09:29:28,942 INFO     [evaluator.py:152] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2025-08-16:09:29:28,942 INFO     [evaluator.py:189] Initializing vllm model, with arguments: {'pretrained': '/opt/app-root/src/granite-3.2-2b-instruct-W4A16', 'add_bos_token': True, 'max_model_len': 4096, 'gpu_memory_utilization': 0.5, 'trust_remote_code': True}
INFO 08-16 09:29:35 [config.py:1604] Using max model len 4096
INFO 08-16 09:29:36

With powerful GPU(s), you could also run the vLLM based evals with the following - using higher GPU memory utilization and chunked prefill. 
```bash
!lm_eval \
  --model vllm \
  --model_args pretrained=$SAVE_DIR,dtype=auto,add_bos_token=True,max_model_len=4096,tensor_parallel_size=1,gpu_memory_utilization=0.8,enable_chunked_prefill=True \
  --trust_remote_code \
  --tasks openllm \
  --write_out \
  --batch_size auto \
  --output_path output_dir \
  --show_config
```

### Upload the optimized model to MinIO

In [19]:
import os
from boto3 import client

current_dir = os.getcwd()
OPTIMIZED_MODEL_DIR = current_dir + "/granite-3.2-2b-instruct-W4A16"
S3_PATH = "granite-int4-notebook"

print('Starting upload of quantizied model')
s3_endpoint_url = os.environ["AWS_S3_ENDPOINT"]
s3_access_key = os.environ["AWS_ACCESS_KEY_ID"]
s3_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
s3_bucket_name = os.environ["AWS_S3_BUCKET"]

print(f'Uploading predictions to bucket {s3_bucket_name} '
        f'to S3 storage at {s3_endpoint_url}')

s3_client = client(
    's3', endpoint_url=s3_endpoint_url, aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key, verify=False
)

# Walk through the local folder and upload files
for root, dirs, files in os.walk(OPTIMIZED_MODEL_DIR):
    for file in files:
        local_file_path = os.path.join(root, file)
        s3_file_path = os.path.join(S3_PATH, local_file_path[len(OPTIMIZED_MODEL_DIR)+1:])
        s3_client.upload_file(local_file_path, s3_bucket_name, s3_file_path)
        print(f'Uploaded {local_file_path}')

print('Finished uploading of quantizied model')

Starting upload of quantizied model
Uploading predictions to bucket models to S3 storage at http://minio-service.minio.svc.cluster.local:9000
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/added_tokens.json
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/model.safetensors
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/special_tokens_map.json
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/generation_config.json
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/tokenizer_config.json
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/vocab.json
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/merges.txt
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/recipe.yaml
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/chat_template.jinja
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/tokenizer.json
Uploaded /opt/app-root/src/granite-3.2-2b-instruct-W4A16/config.json
Finished uploading of quantizied model


### Bonus labs
- Experiment with different quantization scheme & method to further improve its accuracy
- Prepare a new dataset tailored to a specific use case by collecting and performing data mixing for calibration